In [ ]:
from langgraph.graph import StateGraph,START,END
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import TypedDict

c:\Users\Admin\Desktop\Codes\Learning-LangGraph\genvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
class Batsman(TypedDict):
    runs:int
    balls:int
    fours:int
    sixes:int

    summary:str
    sr:float 
    bpb:float
    boundary_percent:float

In [15]:
def calculate_sr(state:Batsman)->Batsman:
    runs = state['runs']
    balls = state['balls']

    sr = (runs/balls)*100

    return {'sr':sr}

In [14]:
def calculate_bpb(state:Batsman)->Batsman:
    
    bpb=state['balls']/(state['fours']+state['sixes'])

    return {'bpb':bpb}

In [16]:
def calculate_bp(state:Batsman)->Batsman:
    bp = (((state['fours'] * 4) + (state['sixes'] * 6))/state['runs'])*100

    return {'boundary_percent':bp}

In [22]:
def summary(state:Batsman)->Batsman:
    summary = f"""Strike Rate = {state['sr']}
                Balls Per Boundary = {state['bpb']}
                Boundary Percent = {state['boundary_percent']}
                """
    
    state['summary'] = summary

    return state
    

In [23]:
graph = StateGraph(Batsman)

graph.add_node('calculate_sr',calculate_sr)
graph.add_node('calculate_bpb',calculate_bpb)
graph.add_node('calculate_bp',calculate_bp)
graph.add_node('summary',summary)



graph.add_edge(START,'calculate_sr')
graph.add_edge(START,'calculate_bpb')
graph.add_edge(START,'calculate_bp')

graph.add_edge('calculate_sr','summary')
graph.add_edge('calculate_bpb','summary')
graph.add_edge('calculate_bp','summary')

graph.add_edge('summary',END)

workflow=graph.compile()

In [24]:
initial_state={'runs':100,'balls':50,'fours':5,'sixes':5}

final_state=workflow.invoke(initial_state)

print(final_state)


{'runs': 100, 'balls': 50, 'fours': 5, 'sixes': 5, 'summary': 'Strike Rate = 200.0\n                Balls Per Boundary = 5.0\n                Boundary Percent = 50.0\n                ', 'sr': 200.0, 'bpb': 5.0, 'boundary_percent': 50.0}
